In [1]:
import cv2
import mediapipe as mp
import math
from djitellopy import tello
import numpy as np
import time

In [2]:
class handDetector():
    def __init__(self,
    static_image_mode=False,
    max_num_hands=2,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5):

        self.mode = static_image_mode
        self.maxHands = max_num_hands
        self.detectionCon = min_detection_confidence
        self.trackCon = min_tracking_confidence
        self.modelComplex = model_complexity
        
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,
        self.modelComplex, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]
 
    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)
    
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                # print(id, cx, cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax
    
            if draw:
                cv2.rectangle(img, (bbox[0] - 20, bbox[1] - 20),
            (bbox[2] + 20, bbox[3] + 20), (0, 255, 0), 2)
        
        return self.lmList, bbox
    
    def fingersUp(self):
        fingers = []
        # Thumb
        if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)
        # 4 Fingers
        for id in range(1, 5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        return fingers
    
    def findDistance(self, p1, p2, img, draw=True):
    
        x1, y1 = self.lmList[p1][1], self.lmList[p1][2]
        x2, y2 = self.lmList[p2][1], self.lmList[p2][2]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        
        if draw:
            cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
            cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
        
        length = math.hypot(x2 - x1, y2 - y1)
        return length, img, [x1, y1, x2, y2, cx, cy]
    


In [3]:
def move(direction,vel,drone):
    leftRight=0
    forwardBackward=0
    upDown=0
    yaw=0
    if(direction=="up"):
        upDown=vel
    elif(direction=="down"):
        upDown=-vel
    elif(direction=="forward"):
        forwardBackward=vel
    elif(direction=="back"):
        forwardBackward=-vel
    elif(direction=="left"):
        yaw=-vel
    elif(direction=="right"):
        yaw=vel
    
    drone.send_rc_control(leftRight,forwardBackward,upDown,yaw)

In [4]:
def codeText(code,drone,is_flying):

    if(code==16):
        if(not is_flying):
            drone.takeoff()
            is_flying= True
        return "Take off"
    elif(code==4):
        return "Do a frontal flip"
    elif(code==28):
        return "Pixelate faces"
    elif(code==24):
        return "Take a picture"
    elif(code==31):
        if(is_flying):
            drone.land()
            is_flying=False
        return "Land"
    elif(code==8):
        return "360º"
    elif(code==12):
        battery = str(drone.get_battery())
        return str("Battery about : " + battery +"%")

In [5]:
pTime = 0
cap = cv2.VideoCapture(0)
cap.set(3, 350)
cap.set(4, 350)
detector = handDetector()
fingersT=[]
fingers_20=[0,0,0,0,0]
counter=0
action="none"

#video
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('tempvideo.mp4', fourcc, 20.0, (300,300))

In [6]:
me=tello.Tello()
me.connect()
print(me.get_battery())

[INFO] tello.py - 107 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 422 - Send command: 'command'
[INFO] tello.py - 446 - Response command: 'ok'


68


In [7]:
me.streamon()

[INFO] tello.py - 422 - Send command: 'streamon'
[INFO] tello.py - 446 - Response streamon: 'ok'


In [8]:
is_flying=False
action=False
code=0
while True:
    frameRead=me.get_frame_read()

    time.sleep(0.05)
    
    if frameRead :
        try:
            img= cv2.resize(frameRead.frame,(300,300))
            img = detector.findHands(img)
            lmList = detector.findPosition(img)
            fingers=detector.fingersUp()
            if(action == False): 
                me.send_rc_control(0,0,0,0)
            
            if(len(fingers)>0):

                if(counter<20):
                    counter= counter + 1
                    bin_finger=str(fingers[0])+str(fingers[1])+str(fingers[2])+str(fingers[3])+str(fingers[4])
                    dec_finger=int(str(bin_finger), 2)
                    fingersT.append(dec_finger)
                else:
                    if(len(fingersT)>1):
                        code= max(set(fingersT),key=fingersT.count)
                        action=codeText(code,me,is_flying)
                    fingersT=[]
                    counter=0
        except Exception as e:
            #print(f'exc: {e}')
            pass
        
        if(action != "none"):
            cv2.line(img, (0, 15), (300, 15), (255, 255, 255), 30)
            cv2.putText(img, str(action) , (10, 18), cv2.FONT_HERSHEY_PLAIN, 1,(40, 40, 40), 2)

            

        cv2.imshow("Image", img)
        out.write(img)
            #print(fingers)
        #if len(lmList) >0:
        #    print(lmList)
   
   
    
    # key controller
    
        #u up       #w forward
        #d down     #s back
        #a left     #d right
        #
    #
     
    
    key = cv2.waitKey(1) & 0xFF    
    
    if key == ord("q"):
        out.release()
        break

    
    elif key == ord("t"):
        action = True
        if( not is_flying):
            me.takeoff()
            is_flying= True
        action= False
    elif key == ord("l"):
        action = True
        if(is_flying):
            me.land()
            is_flying=False
        action = False
            
    elif key == ord('u'):
        move('up',20,me)
    elif key == ord('j'):
        move('down',20,me) 
    elif key == ord("w"):
        move('forward',20,me)
    elif key == ord('a'):
        move('left',20,me)
    elif key == ord('s'):
        move('back',20,me)
    elif key == ord("d"):
        move('right',20,me)

    if key == ord("q"):
        break
out.release()
if(me.is_flying):
    me.land()
me.streamoff()
cv2.destroyAllWindows()

[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 455 - Send command (

Exception: Command 'land' was unsuccessful for 4 tries. Latest response:	'error'

: 